## Create Workshop Notebooks and Users

In [ ]:
import boto3
import botocore
import json
import time
import os
import base64
import docker
import pandas as pd

import project_path # path to helper methods
from lib import workshop
from botocore.exceptions import ClientError

cfn = boto3.client('cloudformation')
iam = boto3.client('iam')

session = boto3.session.Session()
region = session.region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')

### [Create S3 Bucket](https://docs.aws.amazon.com/AmazonS3/latest/gsg/CreatingABucket.html)

We will create an S3 bucket that will be used throughout the workshop for storing our data.

[s3.create_bucket](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_bucket) boto3 documentation

In [ ]:
bucket = workshop.create_bucket_name('workshop-')
session.resource('s3').create_bucket(Bucket=bucket, CreateBucketConfiguration={'LocationConstraint': region})
print(bucket)

### Create the IAM Users and SageMaker Notebooks

We will use CloudFormation to create the workshop users and SageMaker Notebooks to run the workshop.

In [ ]:
!cat ../research-env.yml

In [ ]:
research_file = 'research-env.yml'
session.resource('s3').Bucket(bucket).Object(os.path.join('cfn', research_file)).upload_file( '../' + research_file)

### Execute CloudFormation Stack to generate Users and Environment

Creates a stack as specified in the template. After the call completes successfully, the stack creation starts. You can check the status of the stack via the DescribeStacks API.

[cfn.create_stack](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/cloudformation.html#CloudFormation.Client.create_stack)

In [ ]:
user_count = 2
cfn_template = 'https://s3-{0}.amazonaws.com/{1}/cfn/{2}'.format(region, bucket, research_file)
print(cfn_template)

for x in range(user_count):
    user_stack_name = 'Workshop-' + str(x + 1)
    response = cfn.create_stack(
        StackName=user_stack_name,
        TemplateURL=cfn_template,
        Capabilities = ["CAPABILITY_NAMED_IAM"],
        Parameters=[
            {
                'ParameterKey': 'NotebookInstanceType',
                'ParameterValue': 'ml.t2.medium'
            },
            {
                'ParameterKey': 'UserName',
                'ParameterValue': 'WorkshopUser' + str(x + 1)
            },
            {
                'ParameterKey': 'IAMUserPassword',
                'ParameterValue': 'WorkshopUser#' + str(x + 1)
            }            
            
        ]
    )

    print(response)

## Clean Up

In [ ]:
for x in range(user_count):
    user_stack_name = 'Workshop-' + str(x + 1)
    response = cfn.delete_stack(StackName=user_stack_name)

In [ ]:
!aws s3 rb s3://$bucket --force  